In [1]:
# ====================================================
# CFG
# ====================================================
class CFG1:
    num_workers=4
    path="/kaggle/input/commonlit-exp051-debertav3large/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    gradient_checkpointing=False
    batch_size=8
    max_len = 1024  # 512
    target_size=2
    fc_dropout=0.2
    target_cols=['content','wording']
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    freezing=True
    freeze_layer=False
    n_freeze_layer=0
    text="text1"
    custommodel="cls"
    
    
    
class CFG2:
    num_workers=4
    path="/kaggle/input/commonlit-exp055-debertav3large/"
    config_path='/kaggle/input/commonlit-exp051-debertav3large/config.pth'
    model="microsoft/deberta-v3-large"
    gradient_checkpointing=False
    batch_size=16
    max_len = 1024  # 512
    target_size=2
    fc_dropout=0.2
    target_cols=['content','wording']
    seed=77
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    freezing=False
    freeze_layer=False
    n_freeze_layer=0
    text="text1"
    custommodel="cls"
    
    
class CFG3:
    num_workers=4
    path="/kaggle/input/commonlit-exp004-v3base/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-base"
    gradient_checkpointing=False
    batch_size=16
    max_len = 512
    target_size=2
    fc_dropout=0.2
    target_cols=['content','wording']
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    freezing=False
    freeze_layer=False
    n_freeze_layer=0
    text="text2"
    custommodel="attention"
    

class CFG4:
    num_workers=4
    path="/kaggle/input/commonlit-exp084-deberta-v3-large/"
    config_path='/kaggle/input/commonlit-exp051-debertav3large/config.pth'
    model="microsoft/deberta-v3-large"
    gradient_checkpointing=False
    batch_size=16
    max_len = 1024  # 512
    target_size=2
    fc_dropout=0.2
    target_cols=['content','wording']
    seed=2023
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    freezing=False
    freeze_layer=True
    n_freeze_layer=4
    text="text1"
    custommodel="cls"
    

    
class CFG5:
    num_workers=4
    path="/kaggle/input/commonlit-exp085-deberta-v3-large/"
    config_path='/kaggle/input/commonlit-exp051-debertav3large/config.pth'
    model="microsoft/deberta-v3-large"
    gradient_checkpointing=False
    batch_size=8
    max_len = 1024  # 512
    target_size=2
    fc_dropout=0.2
    target_cols=['content','wording']
    seed=2023
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    freezing=False
    freeze_layer=True
    n_freeze_layer=6
    text="text1"
    custommodel="cls"

In [2]:
!pip install "/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

Processing /kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl


In [3]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import stopwords
from collections import Counter
import spacy
from spellchecker import SpellChecker
import lightgbm as lgb
from catboost import Pool, CatBoostRegressor

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
%env TOKENIZERS_PARALLELISM=false


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.13.3
transformers.__version__: 4.30.2
env: TOKENIZERS_PARALLELISM=false


In [4]:
# ====================================================
# tokenizer
# ====================================================
# CFG1.tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/commonlit-exp044-deberta-v3-large/tokenizer/')
CFG2.tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/commonlit-exp044-deberta-v3-large/tokenizer/')
CFG3.tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/commonlit-deberta-train/tokenizer/')
CFG4.tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/commonlit-exp044-deberta-v3-large/tokenizer/')
#CFG5.tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/commonlit-exp044-deberta-v3-large/tokenizer/')

In [5]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename='inference'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG1.seed)

In [6]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    
    for parameter in module.parameters():
        parameter.requires_grad = False

In [7]:
"""
# ====================================================
# oof
# ====================================================
oof_df1 = pd.read_pickle(CFG1.path+'oof_df.pkl')
oof_df2 = pd.read_pickle(CFG2.path+'oof_df.pkl')
oof_df4 = pd.read_pickle(CFG4.path+'oof_df.pkl')
oof_df5 = pd.read_pickle(CFG5.path+'oof_df.pkl')

lgbm_oof_df1 = pd.read_csv("/kaggle/input/commonlit-lgbm-prediction/oof_v3base_lgbm.csv")
cat_oof_df1 = pd.read_csv("/kaggle/input/commonlit-catboost-prediction/oof_v3base_cat.csv")

oof_df1.rename(columns={'pred_content':'pred_content1', 'pred_wording':'pred_wording1'},inplace=True)
oof_df2.rename(columns={'pred_content':'pred_content2', 'pred_wording':'pred_wording2'},inplace=True)
oof_df4.rename(columns={'pred_content':'pred_content4', 'pred_wording':'pred_wording4'},inplace=True)
oof_df5.rename(columns={'pred_content':'pred_content5', 'pred_wording':'pred_wording5'},inplace=True)

lgbm_oof_df1.rename(columns={"content_lgbm_pred":"pred_content_lgbm", "wording_lgbm_pred":"pred_wording_lgbm"},inplace=True)
cat_oof_df1.rename(columns={"content_cat_pred":"pred_content_cat", "wording_cat_pred":"pred_wording_cat"},inplace=True)

oof_df = oof_df1.merge(oof_df2[["student_id", "pred_content2", "pred_wording2"]],how="left",on="student_id")\
.merge(oof_df4[["student_id", "pred_content4", "pred_wording4"]],how="left",on="student_id").merge(oof_df5[["student_id", "pred_content5", "pred_wording5"]],how="left",on="student_id")\
.merge(lgbm_oof_df1[["student_id","pred_content_lgbm","pred_wording_lgbm"]], how="left", on="student_id").merge(cat_oof_df1[["student_id","pred_content_cat","pred_wording_cat"]], how="left", on="student_id")


for col in CFG1.target_cols:
    #oof_df[f"oof_pred_{col}"] = (oof_df[f"pred_{col}1"]*.25+oof_df[f"pred_{col}2"]*.25+oof_df[f"pred_{col}4"]*.35+oof_df[f"pred_{col}5"]*.15)*.8 + (oof_df[f"pred_{col}_lgbm"]*.5+oof_df[f"pred_{col}_cat"]*.5)*.2
    oof_df[f"oof_pred_{col}"] = (oof_df[f"pred_{col}2"]*.4+oof_df[f"pred_{col}4"]*.6)*.7 + oof_df[f"pred_{col}_lgbm"]*.3
    
labels = oof_df[CFG1.target_cols].values
preds = oof_df[[f"oof_pred_{c}" for c in CFG1.target_cols]].values
score, scores = get_score(labels, preds)
LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')

# Score: 0.4777  Scores: [0.410089229181465, 0.5453701937221018]
"""

'\n# ====================================================\n# oof\n# ====================================================\noof_df1 = pd.read_pickle(CFG1.path+\'oof_df.pkl\')\noof_df2 = pd.read_pickle(CFG2.path+\'oof_df.pkl\')\noof_df4 = pd.read_pickle(CFG4.path+\'oof_df.pkl\')\noof_df5 = pd.read_pickle(CFG5.path+\'oof_df.pkl\')\n\nlgbm_oof_df1 = pd.read_csv("/kaggle/input/commonlit-lgbm-prediction/oof_v3base_lgbm.csv")\ncat_oof_df1 = pd.read_csv("/kaggle/input/commonlit-catboost-prediction/oof_v3base_cat.csv")\n\noof_df1.rename(columns={\'pred_content\':\'pred_content1\', \'pred_wording\':\'pred_wording1\'},inplace=True)\noof_df2.rename(columns={\'pred_content\':\'pred_content2\', \'pred_wording\':\'pred_wording2\'},inplace=True)\noof_df4.rename(columns={\'pred_content\':\'pred_content4\', \'pred_wording\':\'pred_wording4\'},inplace=True)\noof_df5.rename(columns={\'pred_content\':\'pred_content5\', \'pred_wording\':\'pred_wording5\'},inplace=True)\n\nlgbm_oof_df1.rename(columns={"conten

In [8]:
"""
for col in CFG1.target_cols:
    oof_df[f"oof_pred_{col}"] = (oof_df[f"pred_{col}1"]*.3+oof_df[f"pred_{col}2"]*.3+oof_df[f"pred_{col}4"]*.4)*.8 + (oof_df[f"pred_{col}_lgbm"]*.5+oof_df[f"pred_{col}_cat"]*.5)*.2

labels = oof_df[CFG1.target_cols].values
preds = oof_df[[f"oof_pred_{c}" for c in CFG1.target_cols]].values
score, scores = get_score(labels, preds)
LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')

# Score: 0.4764  Scores: [0.4099805570720724, 0.542862157384927]
"""

'\nfor col in CFG1.target_cols:\n    oof_df[f"oof_pred_{col}"] = (oof_df[f"pred_{col}1"]*.3+oof_df[f"pred_{col}2"]*.3+oof_df[f"pred_{col}4"]*.4)*.8 + (oof_df[f"pred_{col}_lgbm"]*.5+oof_df[f"pred_{col}_cat"]*.5)*.2\n\nlabels = oof_df[CFG1.target_cols].values\npreds = oof_df[[f"oof_pred_{c}" for c in CFG1.target_cols]].values\nscore, scores = get_score(labels, preds)\nLOGGER.info(f\'Score: {score:<.4f}  Scores: {scores}\')\n\n# Score: 0.4764  Scores: [0.4099805570720724, 0.542862157384927]\n'

In [9]:
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summary_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
submission = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv')

print(f"Prompts Test shape: {prompts_test.shape}")
display(prompts_test.head())
print(f"Summary Test shape: {summary_test.shape}")
display(summary_test.head())
print(f"Submission shape: {submission.shape}")
display(submission.head())

Prompts Test shape: (2, 4)


,prompt_id,prompt_question,prompt_title,prompt_text
0,abc123,Summarize...,Example Title 1,Heading\nText...
1,def789,Summarize...,Example Title 2,Heading\nText...


Summary Test shape: (4, 3)


,student_id,prompt_id,text
0,000000ffffff,abc123,Example text 1
1,111111eeeeee,def789,Example text 2
2,222222cccccc,abc123,Example text 3
3,333333dddddd,def789,Example text 4


Submission shape: (4, 3)


,student_id,content,wording
0,000000ffffff,0.0,0.0
1,111111eeeeee,0.0,0.0
2,222222cccccc,0.0,0.0
3,333333dddddd,0.0,0.0


In [10]:
test = prompts_test.merge(summary_test, on="prompt_id")
display(test.head())

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text
0,abc123,Summarize...,Example Title 1,Heading\nText...,000000ffffff,Example text 1
1,abc123,Summarize...,Example Title 1,Heading\nText...,222222cccccc,Example text 3
2,def789,Summarize...,Example Title 2,Heading\nText...,111111eeeeee,Example text 2
3,def789,Summarize...,Example Title 2,Heading\nText...,333333dddddd,Example text 4


In [11]:
def len2text(x:str):
    if len(x)<200: return 'Quite short'
    if len(x)<300: return 'Short'
    if len(x)<500: return 'Middle'
    else: 
        return 'Long'
    
test["text_length"] = test['text'].apply(lambda x: len2text(x))

In [12]:
SEP = CFG2.tokenizer.sep_token
#test['full_text'] = test['prompt_title'] + SEP + test["text_length"] + SEP + test['prompt_question'] + SEP + test['text']
test['full_text1'] = test['text'] + " " + test['prompt_question'] + " " + test["prompt_text"]
test['full_text2'] = test['prompt_title'] + SEP + test["text_length"] + SEP + test['prompt_question'] + SEP + test['text']

In [13]:
# sort by length to speed up inference
test['tokenize_length'] = [len(CFG2.tokenizer(text)['input_ids']) for text in test['full_text1'].values]
test = test.sort_values('tokenize_length', ascending=True).reset_index(drop=True)
display(test.head())

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,text_length,full_text1,full_text2,tokenize_length
0,abc123,Summarize...,Example Title 1,Heading\nText...,000000ffffff,Example text 1,Quite short,Example text 1 Summarize... Heading\nText...,Example Title 1[SEP]Quite short[SEP]Summarize....,14
1,abc123,Summarize...,Example Title 1,Heading\nText...,222222cccccc,Example text 3,Quite short,Example text 3 Summarize... Heading\nText...,Example Title 1[SEP]Quite short[SEP]Summarize....,14
2,def789,Summarize...,Example Title 2,Heading\nText...,111111eeeeee,Example text 2,Quite short,Example text 2 Summarize... Heading\nText...,Example Title 2[SEP]Quite short[SEP]Summarize....,14
3,def789,Summarize...,Example Title 2,Heading\nText...,333333dddddd,Example text 4,Quite short,Example text 4 Summarize... Heading\nText...,Example Title 2[SEP]Quite short[SEP]Summarize....,14


In [14]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=cfg.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        if self.cfg.text=="text1":
            self.texts = df['full_text1'].values
        else:
            self.texts = df["full_text2"].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs
    
    
def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

In [15]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())
            
        if cfg.freeze_layer:
            for k, param in self.model.base_model.encoder.layer.named_parameters():
                l = int(k.split(".")[0])
                if l < cfg.n_freeze_layer:
                    param.requires_grad = False

        #self.pool = MeanPooling()
        self.output = nn.Sequential(
            nn.LayerNorm(self.config.hidden_size),
            nn.Linear(self.config.hidden_size, self.cfg.target_size)
        )

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        feature = outputs[0][:, 0, :]
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.output(feature)
        return output

In [16]:
# ====================================================
# Model
# ====================================================
class CustomModel2(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
            
        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())
            
        #self.pool = MeanPooling()
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        #feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

In [17]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [18]:
def pred_func(CFG):
    test_dataset = TestDataset(CFG, test)
    test_loader = DataLoader(test_dataset,
                             batch_size=CFG.batch_size,
                             shuffle=False,
                             #collate_fn=collator,
                             num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    predictions = []
    for fold in CFG.trn_fold:
        if CFG.custommodel=="cls":
            model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
        else:
            model = CustomModel2(CFG, config_path=CFG.config_path, pretrained=False)
        state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                           map_location=torch.device('cpu'))
        model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        predictions.append(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    predictions = np.mean(predictions, axis=0)
    return predictions

In [19]:
#predictions1 = pred_func(CFG1)
predictions2 = pred_func(CFG2)
predictions4 = pred_func(CFG4)
#predictions5 = pred_func(CFG5)

bert_predictions = predictions2*.4+predictions4*.6

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connectio

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/pytho

  0%|          | 0/1 [00:00<?, ?it/s]

...

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/pytho

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times


  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/pytho

In [20]:
bert_predictions

array([[-0.34466588, -0.27680224],
       [-0.34880137, -0.28263867],
       [-0.3509633 , -0.28397715],
       [-0.3460034 , -0.28648475]], dtype=float32)

In [21]:
class Preprocessor:
    def __init__(self, 
                # model_name: str,
                ) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(f"/kaggle/input/huggingfacedebertav3variants/deberta-v3-base")
        self.STOP_WORDS = set(stopwords.words('english'))
        
        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.speller = SpellChecker() #Speller(lang='en')
        
    def count_text_length(self, df: pd.DataFrame, col:str) -> pd.Series:
        """ text length """
        tokenizer=self.tokenizer
        return df[col].progress_apply(lambda x: len(tokenizer.encode(x)))

    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """        
        def check_is_stop_word(word):
            return word in self.STOP_WORDS
        
        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))
            
    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int):
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)

        # # Optionally, you can get the frequency of common n-grams for a more nuanced analysis
        # original_ngram_freq = Counter(ngrams(original_words, n))
        # summary_ngram_freq = Counter(ngrams(summary_words, n))
        # common_ngram_freq = {ngram: min(original_ngram_freq[ngram], summary_ngram_freq[ngram]) for ngram in common_ngrams}

        return len(common_ngrams)
    
    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)
        
        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))
        
        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}

    
    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):
        
        wordlist=text.split()
        amount_miss = len(list(self.speller.unknown(wordlist)))

        return amount_miss
    
    def run(self, 
            input_df: pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:
        
        # before merge preprocess
        input_df["prompt_length"] = input_df["prompt_text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        input_df["prompt_tokens"] = input_df["prompt_text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x), 
                skip_special_tokens=True
            )
        )

        input_df["summary_length"] = input_df["text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        input_df["summary_tokens"] = input_df["text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x), 
                skip_special_tokens=True
            )

        )
        input_df["splling_err_num"] = input_df["text"].apply(self.spelling)

        # merge prompts and summaries
        # input_df = summaries.merge(prompts, how="left", on="prompt_id")

        # after merge preprocess
        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']
        
        input_df['word_overlap_count'] = input_df.apply(self.word_overlap_count, axis=1)
        input_df['bigram_overlap_count'] = input_df.apply(
            self.ngram_co_occurrence,args=(2,), axis=1 
        )
        input_df['bigram_overlap_ratio'] = input_df['bigram_overlap_count'] / (input_df['summary_length'] - 1)
        
        input_df['trigram_overlap_count'] = input_df.apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )
        
        input_df['trigram_overlap_ratio'] = input_df['trigram_overlap_count'] / (input_df['summary_length'] - 2)
        
        input_df['quotes_count'] = input_df.apply(self.quotes_count, axis=1)
        
        return input_df.drop(columns=["summary_tokens", "prompt_tokens"])
    
preprocessor = Preprocessor()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
gbdt_test = test.copy()
gbdt_test = preprocessor.run(gbdt_test, mode="test")

gbdt_test.head()

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,text_length,full_text1,full_text2,tokenize_length,prompt_length,summary_length,splling_err_num,length_ratio,word_overlap_count,bigram_overlap_count,bigram_overlap_ratio,trigram_overlap_count,trigram_overlap_ratio,quotes_count
0,abc123,Summarize...,Example Title 1,Heading\nText...,000000ffffff,Example text 1,Quite short,Example text 1 Summarize... Heading\nText...,Example Title 1[SEP]Quite short[SEP]Summarize....,14,7,5,0,0.714286,0,0,0.0,0,0.0,0
1,abc123,Summarize...,Example Title 1,Heading\nText...,222222cccccc,Example text 3,Quite short,Example text 3 Summarize... Heading\nText...,Example Title 1[SEP]Quite short[SEP]Summarize....,14,7,5,0,0.714286,0,0,0.0,0,0.0,0
2,def789,Summarize...,Example Title 2,Heading\nText...,111111eeeeee,Example text 2,Quite short,Example text 2 Summarize... Heading\nText...,Example Title 2[SEP]Quite short[SEP]Summarize....,14,7,5,0,0.714286,0,0,0.0,0,0.0,0
3,def789,Summarize...,Example Title 2,Heading\nText...,333333dddddd,Example text 4,Quite short,Example text 4 Summarize... Heading\nText...,Example Title 2[SEP]Quite short[SEP]Summarize....,14,7,5,0,0.714286,0,0,0.0,0,0.0,0


In [23]:
len2text_dict = {
    "Quite short" : 0,
    "Short" : 1,
    "Middle" : 2,
    "Long" : 3
}


gbdt_test["text_length"] = gbdt_test["text_length"].map(len2text_dict)

In [24]:
gbdt_oof_predictions = pred_func(CFG3)

  0%|          | 0/1 [00:00<?, ?it/s]

Exception ignored in: <function _ConnectionBase.__del__ at 0x788d6bcdf9a0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
V

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "

In [25]:
gbdt_test["pred_content"] = gbdt_oof_predictions[:, 0]
gbdt_test["pred_wording"] = gbdt_oof_predictions[:, 1]

In [26]:
features = [
 'text_length',
 'pred_content',
 'pred_wording',
 'prompt_length',
 'summary_length',
 'splling_err_num',
 'length_ratio',
 'word_overlap_count',
 'bigram_overlap_count',
 'bigram_overlap_ratio',
 'trigram_overlap_count',
 'trigram_overlap_ratio',
 'quotes_count'
]

targets = ["content", "wording"]


categorical_features = ["text_length"]

In [27]:
pred_lgbm_dict = {}
# pred_cat_dict = {}
for target in targets:
    preds_lgbm = []
    # preds_cat = []

    for fold in CFG3.trn_fold:
        # ilocで取り出す行を指定
        X_eval_cv = gbdt_test[features]
        
        model_lgbm = pickle.load(open(f'/kaggle/input/commonlit-lgbm-prediction/debertav3pred_{target}_fold{fold}.pkl', 'rb'))
        # model_cat = pickle.load(open(f'/kaggle/input/commonlit-catboost-prediction/debertav3catpred_{target}_fold{fold}.pkl', 'rb'))
        
        pred_lgbm = model_lgbm.predict(X_eval_cv)
        # pred_cat = model_cat.predict(Pool(X_eval_cv, cat_features=categorical_features))
        preds_lgbm.append(pred_lgbm)
        # preds_cat.append(pred_cat)
    
    pred_lgbm_dict[target] = preds_lgbm
    # pred_cat_dict[target] = preds_cat

In [28]:
for target in targets:
    lgbm_preds = pred_lgbm_dict[target]
    # cat_preds = pred_cat_dict[target]
    for i, lgbm_pred in enumerate(lgbm_preds):
        gbdt_test[f"{target}_lgbm_pred_{i}"] = lgbm_pred
    # for j, cat_pred in enumerate(cat_preds):
     #    gbdt_test[f"{target}_cat_pred_{j}"] = cat_pred

    gbdt_test[f"{target}_lgbm"] = gbdt_test[[f"{target}_lgbm_pred_{fold}" for fold in range(CFG3.n_fold)]].mean(axis=1)
    # gbdt_test[f"{target}_cat"] = gbdt_test[[f"{target}_cat_pred_{fold}" for fold in range(CFG3.n_fold)]].mean(axis=1)

In [29]:
lgbm_predictions = np.array(gbdt_test[[f"{col}_lgbm" for col in CFG3.target_cols]].values)
# cat_predictions = np.array(gbdt_test[[f"{col}_cat" for col in CFG3.target_cols]].values)

gbdt_predictions = lgbm_predictions

In [30]:
gbdt_predictions

array([[-1.45566741, -1.25090018],
       [-1.45566741, -1.24865493],
       [-1.45566741, -1.25090018],
       [-1.45566741, -1.25090018]])

In [31]:
predictions = bert_predictions*.7 + gbdt_predictions*.3

In [32]:
test[CFG1.target_cols] = predictions
submission = submission.drop(columns=CFG1.target_cols).merge(test[['student_id'] + CFG1.target_cols], on='student_id', how='left')
display(submission.head())
submission[['student_id'] + CFG1.target_cols].to_csv('submission.csv', index=False)

,student_id,content,wording
0,000000ffffff,-0.677966,-0.569032
1,111111eeeeee,-0.682375,-0.574054
2,222222cccccc,-0.680861,-0.572444
3,333333dddddd,-0.678903,-0.575809
